In [1]:
# Extract all frames from a video (same dir as this notebook)
# Output goes to: frames_<video_name_without_ext>
# Requires: opencv-python, tqdm

from pathlib import Path
from datetime import timedelta
import cv2
from tqdm import tqdm

# --- config (edit if you want) ---
vname = "20250826_31302PMByGPSMapCamera.mp4"  # video file in the same folder as the notebook
fmt = "jpg"                                   # 'jpg' or 'png'
jpg_quality = 95                              # used if fmt == 'jpg'
# ---------------------------------

vpath = Path(vname)
if not vpath.exists():
    raise FileNotFoundError(f"Video not found: {vpath.resolve()}")

out_dir = Path(f"frames_{vpath.stem}")
out_dir.mkdir(parents=True, exist_ok=True)

cap = cv2.VideoCapture(str(vpath))
if not cap.isOpened():
    raise RuntimeError("Could not open video. Check codec and file integrity.")

fps = cap.get(cv2.CAP_PROP_FPS)
n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"[info] video: {vpath.name}")
print(f"[info] size: {w}x{h}, fps: {fps:.3f}, frames (reported): {n}")

# progress bar set to total if known
pbar = tqdm(total=n if n > 0 else None, desc="Extracting frames", unit="frame")

saved = 0
# Prefer a for-loop: if frame count known, iterate that many times; otherwise, iterate large and break on EOF
limit = n if n > 0 else 10**12

# Encode params for JPEG/PNG
enc_params = []
if fmt.lower() == "jpg":
    enc_params = [int(cv2.IMWRITE_JPEG_QUALITY), int(jpg_quality)]
elif fmt.lower() == "png":
    # 0 (fastest, larger) .. 9 (slowest, smaller); default 3-6 is fine
    enc_params = [int(cv2.IMWRITE_PNG_COMPRESSION), 3]
else:
    raise ValueError("fmt must be 'jpg' or 'png'.")

for i in range(limit):
    ok, frame = cap.read()
    if not ok:
        # end of file or read error
        break

    fn = out_dir / f"frame_{i:06d}.{fmt}"
    # Write image
    cv2.imwrite(str(fn), frame, enc_params)
    saved = saved + 1  # (avoid i += 1)

    # update progress
    pbar.update(1)

pbar.close()
cap.release()

# Summary (duration is approximate if fps unknown)
dur_sec = saved / fps if fps and fps > 0 else 0
print(f"[done] saved: {saved} frames -> {out_dir}")
print(f"[done] approx duration: {str(timedelta(seconds=dur_sec))}")


Extracting frames:   2%|▉                                                           | 3/193 [00:00<00:07, 26.09frame/s]

[info] video: 20250826_31302PMByGPSMapCamera.mp4
[info] size: 1600x1200, fps: 19.060, frames (reported): 193


Extracting frames: 100%|██████████████████████████████████████████████████████████| 193/193 [00:05<00:00, 36.04frame/s]

[done] saved: 193 frames -> frames_20250826_31302PMByGPSMapCamera
[done] approx duration: 0:00:10.126122


In [2]:
from pathlib import Path
import cv2
import re
import pytesseract

# bottom overlay path
p = Path(r"E:\Road_Quality\road_split_out\frame_000006_bottom.jpg")

im = cv2.imread(str(p))
if im is None:
    raise FileNotFoundError(p)

# --- Preprocess (OpenCV) ---
h, w = im.shape[:2]
x_cut = int(w * 0.28)  # crop left mini-map
roi = im[:, x_cut:, :]

roi = cv2.resize(roi, None, fx=1.6, fy=1.6, interpolation=cv2.INTER_CUBIC)
g = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
g = clahe.apply(g)
se = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
g = cv2.morphologyEx(g, cv2.MORPH_TOPHAT, se)
bw = cv2.threshold(g, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# --- OCR ---
cfg = "--oem 3 --psm 6 -l eng"
txt = pytesseract.image_to_string(bw, config=cfg)

# --- Extract lat/long ---
# match things like: Lat27.55995Long-978783519
pattern = re.search(r"Lat\s*([0-9\.\-]+)[^\d\-]+Long\s*([0-9\.\-]+)", txt, re.IGNORECASE)
if pattern:
    lat, lon = pattern.groups()
    print(f"{lat},{lon}")
else:
    print("Could not parse lat/long")


27.55995,-97878351
